# Smart contract analysis
Analyse smart contracts from a MongoDB. To load data into MongoDB use the data-loader package.

In [3]:
import pymongo
from pprint import pprint

In [4]:
db = pymongo.MongoClient()

## Exploration

In [6]:
db.code.collection_names()

[u'code_base']

In [7]:
pprint(
    db.code.code_base.find_one()
)

{u'_id': 1476,
 u'code': {u'body': [{u'end': 48,
                      u'end_version': None,
                      u'start': 25,
                      u'start_version': {u'operator': u'^',
                                         u'type': u'VersionLiteral',
                                         u'version': u'0.4.4'},
                      u'type': u'PragmaStatement'},
                     {u'body': [{u'end': 91,
                                 u'is_constant': False,
                                 u'literal': {u'array_parts': [],
                                              u'end': 77,
                                              u'literal': u'address',
                                              u'members': [],
                                              u'start': 70,
                                              u'type': u'Type'},
                                 u'name': u'owner',
                                 u'start': 70,
                                 u'type': u'S